In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

### Add data to counts

In [199]:
counts = pd.read_csv("../../resources/final_counts.txt", sep="\t")
counts.shape

(16387, 181)

In [200]:
bouake_counts = pd.read_csv("../../../rna-seq-bouake/results/quant/rawcounts.tsv", sep="\t")
bouake_counts.columns = bouake_counts.columns.str.replace("Anogam_", " Bouake_").str.replace("MC", "col_unexp").str.replace("MS", "col_exp").str.replace("SC", "gamb_unexp").str.replace("SS", "gamb_exp")
bouake_counts.head(3)

,GeneID,Bouake_Ki11,Bouake_Ki2,Bouake_Ki5,Bouake_Ki7,Bouake_col_unexp6,Bouake_col_unexp7,Bouake_col_unexp8,Bouake_col_unexp9,Bouake_col_exp1,...,Bouake_Ng4,Bouake_Ng7,Bouake_gamb_unexp10,Bouake_gamb_unexp6,Bouake_gamb_unexp7,Bouake_gamb_unexp9,Bouake_gamb_exp11,Bouake_gamb_exp5,Bouake_gamb_exp7,Bouake_gamb_exp9
0,AGAP000002,5686,5634,7127,5494,5470,3016,6043,4496,4789,...,4811,3718,4628,6639,3164,5128,6471,4439,4095,4575
1,AGAP000005,10900,10540,13624,10039,8295,3185,8557,7962,5677,...,8562,6951,5700,8411,3102,7615,9637,5501,6589,8318
2,AGAP000007,4195,4428,4815,3950,3660,1844,3639,4189,2267,...,3390,2729,3204,4084,2350,4416,4344,3309,3242,3779


In [201]:
busia_counts = pd.read_csv("../../../rna-seq-busia/results/quant/rawcounts.tsv", sep="\t")
busia_counts.columns = busia_counts.columns.str.replace("G24-", "").str.replace("G28-", "")
busia_counts.head(3)

,GeneID,BusiaSurvivors1,BusiaSurvivors2,BusiaSurvivors3,BusiaSurvivors4,BusiaSurvivors5,BusiaSurvivors6,BusiaParental1,BusiaParental2,BusiaParental3,BusiaParental4,BusiaParental5,BusiaParental6,Kisumu1,Kisumu2,Kisumu3,Kisumu4
0,AGAP000002,5454,5744,4947,5720,6411,5964,5110,5781,5645,5935,6323,5656,4001,3905,3712,4173
1,AGAP000005,9109,10237,8409,9858,12172,10570,9948,7926,7751,10022,9436,8272,5423,5431,4938,5914
2,AGAP000007,4221,4312,3682,4035,4149,3966,3474,4368,3863,4045,3562,4030,3693,3443,3418,3374


In [202]:
abo_counts = pd.read_csv("../../../rna-seq-abo-tiass/results/quant/rawcounts.tsv", sep="\t")
genes = abo_counts['GeneID'] # remove aboisso as contaminated
abo_counts = pd.concat([genes, abo_counts.iloc[:, 13:]], axis=1) # remove aboisso as contaminated

ng_counts = bouake_counts.filter(like="Ng")
ng_counts.columns = ng_counts.columns.str.replace("Bouake", "CDI")

abo_counts = pd.concat([abo_counts, ng_counts], axis=1)
abo_counts.head(2)

,GeneID,ContTia1,ContTia2,ContTia4,DeltTia1,DeltTia2,DeltTia5,MalaTia1,MalaTia2,MalaTia4,PiriTia2,PiriTia3,PiriTia4,CDI_Ng1,CDI_Ng11,CDI_Ng4,CDI_Ng7
0,AGAP000002,482,827,785,819,776,981,767,907,946,481,575,813,5441,5123,4811,3718
1,AGAP000005,1130,1688,1672,1594,1717,1812,1671,1856,1825,842,1135,1296,10638,9715,8562,6951


#### Lets merge these counts

In [203]:
counts = counts.merge(bouake_counts).merge(abo_counts).merge(busia_counts)
counts.shape

(16387, 237)

#### Now we must update col_data

In [283]:
metadata = pd.read_csv("../../config/ALLcoldata.txt", sep="\t")
#metadata = metadata[['colData', 'condition', 'resistance', 'species', 'batch']]
meta.loc[:,'condition'] = meta['condition'].str.replace("_fun", "")
metadata.head(3)

,colData,condition,resistance,species,condition2,batch,condition3
0,Tie1,Tiefora,resistant,gambiae,Tiefora,1,Tiefora
1,Tie2,Tiefora,resistant,gambiae,Tiefora,1,Tiefora
2,Tie3,Tiefora,resistant,gambiae,Tiefora,1,Tiefora


In [284]:
bouake_df = pd.DataFrame(bouake_counts.iloc[:, 1:].columns, columns=['colData'])
bouake_df.loc[:, 'condition'] = bouake_df.loc[:, 'colData'].str.rstrip('0123456789') 
bouake_df.loc[:, 'resistance'] = np.hstack([np.repeat("susceptible", 4), 
                                           np.repeat("resistant", 8), 
                                           np.repeat("susceptible", 4), 
                                           np.repeat("resistant", 8)])
bouake_df.loc[:, 'species'] = np.repeat("gambiae", 24)
bouake_df.loc[:, 'batch'] = np.hstack([np.repeat(10, 4), 
                                       np.repeat(11, 12), 
                                       np.repeat(10, 8)])

In [285]:
busia_df = pd.DataFrame(busia_counts .iloc[:, 1:].columns, columns=['colData'])
busia_df.loc[:, 'condition'] = busia_df.loc[:, 'colData'].str.rstrip('0123456789')
busia_df.loc[:, 'resistance'] = np.hstack([np.repeat("resistant", 12), np.repeat("susceptible", 4)])
busia_df.loc[:, 'species'] = np.repeat("gambiae", 16)
busia_df.loc[:, 'batch'] = 12

In [286]:
abo_df = pd.DataFrame(abo_counts .iloc[:, 1:].columns, columns=['colData'])
abo_df.loc[:, 'condition'] = abo_df.loc[:, 'colData'].str.rstrip('0123456789')
abo_df.loc[:, 'resistance'] = np.hstack([np.repeat("resistant", 12), np.repeat("susceptible", 4)])
abo_df.loc[:, 'species'] = np.repeat("gambiae", 16)
abo_df.loc[:, 'batch'] = 13

In [287]:
metadata = pd.concat([metadata, bouake_df, abo_df, busia_df])

### Write to file

In [288]:
counts.to_csv("../../results/final_raw_counts.tsv", sep="\t", index=None)
metadata.to_csv("../../config/sample_metadata.tsv", sep="\t", index=None)

### Create metadata

In [230]:
def load_species(df, meta):
    spp=np.array([])
    for c in df.resistant:
        spps = meta.query(f"condition == '{c}'")['species'].unique()
        spp = np.append(spp, spps)
    return spp

In [303]:
metadata = pd.read_csv("../../config/sample_metadata.tsv", sep="\t")
meta = metadata[['condition', 'species']].drop_duplicates()
meta.loc[:, 'condition'] = meta.loc[:, 'condition'].str.lstrip(" ")

In [315]:
fc_data = pd.read_csv("../../results/fc_data.tsv", sep="\t")
fc_data = fc_data.iloc[:, 1:-2]

comp_df = pd.DataFrame(fc_data.columns.str.replace("_log2FoldChange", ""), columns=['comparison'])

comp_df.loc[:, 'resistant'] = comp_df.loc[:, 'comparison'].str.split("_v_").str.get(0)
comp_df.loc[:, 'susceptible'] = comp_df.loc[:, 'comparison'].str.split("_v_").str.get(-1)
comp_df.loc[:, 'susceptible'] = comp_df.loc[:, 'susceptible'].str.replace("Bouake_", "")
comp_df.loc[:, 'species'] = load_species(comp_df, meta)

In [316]:
comp_df

,comparison,resistant,susceptible,species
0,Tiefora_v_NG,Tiefora,NG,gambiae
1,Gou_v_Moz,Gou,Moz,arabiensis
2,Ban_v_BanS,Ban,BanS,gambiae
3,BanRe_v_BanS,BanRe,BanS,gambiae
4,Bak_v_Kis,Bak,Kis,gambiae
5,VK7_v_Kis,VK7,Kis,gambiae
6,Cameroon_v_NG,Cameroon,NG,gambiae
7,Chad_v_NG,Chad,NG,gambiae
8,Niger_v_NG,Niger,NG,gambiae
9,Nigeria_v_NG,Nigeria,NG,gambiae


In [321]:
comp_df.loc[:, 'country'] = ['Burkina Faso', 'Burkina Faso', 'Burkina Faso', 'Burkina Faso', 'Burkina Faso', 
                             'Burkina Faso', 'Cameroon', 'Chad', 'Niger', 'Nigeria', 
                             "Cote d'Ivoire", "Cote d'Ivoire", "Cote d'Ivoire", "Cote d'Ivoire", "Cote d'Ivoire",
                             "Cote d'Ivoire", 'Cameroon', 'Mozambique', 'Ghana', 'Malawi',
                             'Uganda', 'Ethiopia', 'Ethiopia', 'Ethiopia', 'Ethiopia'] + np.repeat("Cote d'Ivoire", 8).tolist() + ["Uganda", "Uganda"]

In [323]:
comp_df.to_csv("../../comparison_metadata.tsv", sep="\t", index=False)

### Get latitude and longitude of locations

In [325]:
comp_df

,comparison,resistant,susceptible,species,country
0,Tiefora_v_NG,Tiefora,NG,gambiae,Burkina Faso
1,Gou_v_Moz,Gou,Moz,arabiensis,Burkina Faso
2,Ban_v_BanS,Ban,BanS,gambiae,Burkina Faso
3,BanRe_v_BanS,BanRe,BanS,gambiae,Burkina Faso
4,Bak_v_Kis,Bak,Kis,gambiae,Burkina Faso
5,VK7_v_Kis,VK7,Kis,gambiae,Burkina Faso
6,Cameroon_v_NG,Cameroon,NG,gambiae,Cameroon
7,Chad_v_NG,Chad,NG,gambiae,Chad
8,Niger_v_NG,Niger,NG,gambiae,Niger
9,Nigeria_v_NG,Nigeria,NG,gambiae,Nigeria
